In [5]:
def parse_data_config(path):
    """Parses the data configuration file"""
    options = dict()
    options['gpus'] = '0,1,2,3'
    options['num_workers'] = '10'
    with open(path, 'r') as fp:
        lines = fp.readlines()
    for line in lines:
        line = line.strip()
        if line == '' or line.startswith('#'):
            continue
        key, value = line.split('=')
        options[key.strip()] = value.strip()
    return options



data_config = parse_data_config('/home/lsc/a409/users/lisuicheng/Machine_learning/PyTorch-YOLOv3/config/coco.data')
train_path = data_config["train"]
train_path

'data/coco/trainvalno5k.txt'

In [13]:
import glob
import random
import os
import numpy as np

import torch

from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import matplotlib.patches as patches

from skimage.transform import resize

class ListDataset(Dataset):
    def __init__(self, list_path, img_size=416):
        with open(list_path, 'r') as file:
            self.img_files = file.readlines()
        self.label_files = [path.replace('images', 'labels').replace('.png', '.txt').replace('.jpg', '.txt') for path in self.img_files]
        self.img_shape = (img_size, img_size)
        self.max_objects = 50

    def __getitem__(self, index):

        #---------
        #  Image
        #---------

        img_path = self.img_files[index % len(self.img_files)].rstrip()
        print(img_path)
        img = np.array(Image.open(img_path))

        # Handles images with less than three channels
        while len(img.shape) != 3:
            index += 1
            img_path = self.img_files[index % len(self.img_files)].rstrip()
            img = np.array(Image.open(img_path))

        h, w, _ = img.shape
        dim_diff = np.abs(h - w)
        # Upper (left) and lower (right) padding
        pad1, pad2 = dim_diff // 2, dim_diff - dim_diff // 2
        # Determine padding
        pad = ((pad1, pad2), (0, 0), (0, 0)) if h <= w else ((0, 0), (pad1, pad2), (0, 0))
        # Add padding
        input_img = np.pad(img, pad, 'constant', constant_values=128) / 255.
        padded_h, padded_w, _ = input_img.shape
        # Resize and normalize
        input_img = resize(input_img, (*self.img_shape, 3), mode='reflect')
        # Channels-first
        input_img = np.transpose(input_img, (2, 0, 1))
        # As pytorch tensor
        input_img = torch.from_numpy(input_img).float()

        #---------
        #  Label
        #---------

        label_path = self.label_files[index % len(self.img_files)].rstrip()

        labels = None
        if os.path.exists(label_path):
            labels = np.loadtxt(label_path).reshape(-1, 5)
            # Extract coordinates for unpadded + unscaled image
            x1 = w * (labels[:, 1] - labels[:, 3]/2)
            y1 = h * (labels[:, 2] - labels[:, 4]/2)
            x2 = w * (labels[:, 1] + labels[:, 3]/2)
            y2 = h * (labels[:, 2] + labels[:, 4]/2)
            # Adjust for added padding
            x1 += pad[1][0]
            y1 += pad[0][0]
            x2 += pad[1][0]
            y2 += pad[0][0]
            # Calculate ratios from coordinates
            labels[:, 1] = ((x1 + x2) / 2) / padded_w
            labels[:, 2] = ((y1 + y2) / 2) / padded_h
            labels[:, 3] *= w / padded_w
            labels[:, 4] *= h / padded_h
        # Fill matrix
        filled_labels = np.zeros((self.max_objects, 5))
        if labels is not None:
            filled_labels[range(len(labels))[:self.max_objects]] = labels[:self.max_objects]
        filled_labels = torch.from_numpy(filled_labels)

        return img_path, input_img, filled_labels

    def __len__(self):
        return len(self.img_files)

In [14]:
a = ListDataset('/home/lsc/a409/users/lisuicheng/Machine_learning/PyTorch-YOLOv3/data/coco/trainvalno5k.txt')

In [15]:
a.label_files

['/home/lsc/a409/users/lisuicheng/machine_learning/PyTorch-YOLOv3/data/coco/labels/train2014/COCO_train2014_000000000009.txt\n',
 '/home/lsc/a409/users/lisuicheng/machine_learning/PyTorch-YOLOv3/data/coco/labels/train2014/COCO_train2014_000000000025.txt\n',
 '/home/lsc/a409/users/lisuicheng/machine_learning/PyTorch-YOLOv3/data/coco/labels/train2014/COCO_train2014_000000000030.txt\n',
 '/home/lsc/a409/users/lisuicheng/machine_learning/PyTorch-YOLOv3/data/coco/labels/train2014/COCO_train2014_000000000034.txt\n',
 '/home/lsc/a409/users/lisuicheng/machine_learning/PyTorch-YOLOv3/data/coco/labels/train2014/COCO_train2014_000000000036.txt\n',
 '/home/lsc/a409/users/lisuicheng/machine_learning/PyTorch-YOLOv3/data/coco/labels/train2014/COCO_train2014_000000000049.txt\n',
 '/home/lsc/a409/users/lisuicheng/machine_learning/PyTorch-YOLOv3/data/coco/labels/train2014/COCO_train2014_000000000061.txt\n',
 '/home/lsc/a409/users/lisuicheng/machine_learning/PyTorch-YOLOv3/data/coco/labels/train2014/COCO

In [16]:
# COCO_train2014_000000000025.txt下的东西
# 猜：应该是分类和bbox的参数
# 23 0.770336 0.489695 0.335891 0.697559 
# 23 0.185977 0.901608 0.206297 0.129554 

# COCO_train2014_00000000XXXX.txt下的东西
# 猜:0应该时分类, 分类时从0开始的
# 0 0.524344 0.487641 0.569271 0.880906 
# 29 0.704594 0.477438 0.079562 0.181938 
# 29 0.685406 0.540164 0.028313 0.096359 
# 29 0.639698 0.502242 0.050937 0.191016 
# 29 0.668979 0.514766 0.021417 0.150844 
# 0 0.974479 0.649781 0.051042 0.063062 
# 0 0.014667 0.345727 0.029333 0.153141 
# 29 0.704656 0.544727 0.039229 0.065234 